In [ ]:
import random
import logging
from collections import Counter
from numpy.random import choice
from scipy.stats import binom
from math import ceil
from typing import Callable
from time import perf_counter

In [ ]:
PROBLEM_SIZE = 500

OFFSPRING_SIZE = 100

POPULATION_SIZE = 10

NUM_GENERATIONS = 50

# DATA USED TO SEE WHICH TYPE OF CROSSOVER WAS WORKING BETTER, UNIFORM WERE WORKING BETTER SO I KEPT IT
#winsuni=0
#winssplit=0

In [ ]:
def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]


In [ ]:
problem_=problem(PROBLEM_SIZE,seed=42)
# MAXREPETITIONS REPRESENTS THE WORST CASE, THIS IS USED TO UNDERSTAND RELATIVELY HOW MANY SETS TO REMOVE EACH TIME
PROBLEM_LEN=len(problem_)
MAXREPETITIONS=sum([len(problem_[ind]) for ind in range(PROBLEM_LEN)])
AVGPERSET=MAXREPETITIONS/PROBLEM_LEN
gen=0
problem_

In [ ]:
def goal_test(genome):
    # IS THE GENOME A VALID SOLUTION? TO DO IT JUST CHECK THE LEN OF THE SET
    return len(set([problem_[i][j] for i in range(PROBLEM_LEN) for j in range(len(problem_[i])) if genome[i]]))==PROBLEM_SIZE

def weight_sol(genome):
    """Returns the weigth of a genome"""
    len_=0
    for i in range(PROBLEM_LEN):
        if genome[i]:
            len_+=len(problem_[i])
    return len_
def solution_sets(genome):
    """Function used to get the sets in a genome so it's used to print genomes at the end"""
    return [problem_[i] for i in range(PROBLEM_LEN) if genome[i]]

def fitness_bonus_sol(genome):
    # len of set that we are generating minus number of repetitions plus bonus if solution, so OPTIMAL SOLUTION has
    # fitness of PROBLEM_SIZE*2, because it would have len of the set as PROBLEM_SIZE(it's a solution so the set would
    # contain each number) then the sum on the dictionary took from the counter of repetitions would be 0, then
    # a bonus of PROBLEM_SIZE because the solution is valid, so PROBLEM_SIZE-0+PROBLEM_SIZE = 2*PROBLEM_SIZE
    return len(set([problem_[i][j] for i in range(PROBLEM_LEN) for j in range(len(problem_[i])) if genome[i]])) -sum({k:v-1 for k,v in dict(Counter([problem_[i][j] for i in range(PROBLEM_LEN) for j in range(len(problem_[i])) if genome[i]])).items()}.values()) + int(goal_test(genome))*PROBLEM_SIZE

def fitness_bonus_genome(genome):
    """This fitness gives less bonus to the solutions comparing it to non solution, in fact a genome that has covered PROBLEM_SIZE-1
    elements will get PROBLEM_SIZE-1 bonus points, so the gap between a solution or not is really tiny, this favors non solutions a lot early on"""
    return 2*len(set([problem_[i][j] for i in range(PROBLEM_LEN) for j in range(len(problem_[i])) if genome[i]])) -sum({k:v-1 for k,v in dict(Counter([problem_[i][j] for i in range(PROBLEM_LEN) for j in range(len(problem_[i])) if genome[i]])).items()}.values())

# WEIGTHS_ROULETTE IS THE DISTRIBUTION USED BEFORE AND AS DEFAULT(BUT NOW I USE A UNIFORM ONE KINDA)
# THIS DISTRIBUTION IS A SIMPLE BINOMIAL ONE THAT IS ON THE BASIS OF THE RANKING, NOW I DON'T USE IT
# BECAUSE WHEN WE HAVE ALL THE POPULATION THAT ARE SIMILAR WE STILL PICK MAINLY THE FIRST 4 INDIVIDUALS 
WEIGHTS_ROULETTE=[binom.pmf(k=_,n=POPULATION_SIZE-1,p=1/POPULATION_SIZE) for _ in range(POPULATION_SIZE)]
#print(f"Weights roulette {WEIGHTS_ROULETTE}, their sum is {sum(WEIGHTS_ROULETTE)}")

def select_parent(pop,weigths=WEIGHTS_ROULETTE,k=2):
    # Using a wheel roulette TO SELECT K PARENTS, THIS IS WITHOUT REPLACEMENT, SO THE PARENT CAN'T BE TAKEN MORE THAN ONCE
    # IN A SINGLE CALL
    return [pop[ind] for ind in choice(range(POPULATION_SIZE),k,p=weigths,replace=False )]

def cross_overint(genome1,genome2):
    """crossover that takes the union of the genomes and takes from that union the average # of sets that the genomes take"""
    list_=[ind for ind in range(PROBLEM_LEN) if genome1[ind] or genome2[ind]]
    choices=choice(list_,(sum(genome1)+sum(genome2))//2,p=[1/len(list_) for _ in range(len(list_))],replace=False) 
    return tuple([True if _ in choices else False for _ in range(PROBLEM_LEN)])

def cross_overuni(genome1,genome2):
    """Uniform crossover, this is the one activated by default, it is weigthed"""
    return tuple([random.choice([genome1[ind],genome1[ind],genome2[ind]]) for ind in range(PROBLEM_LEN)])


def cross_oversplit(genome1,genome2):
    """One point split crossover"""
    point = random.randint(0,PROBLEM_LEN-1)
    return genome1[:point] + genome2[point:] if random.choice([True,False]) else genome2[:point] + genome1[point:]

"""def covered_elements(genome):
    return len(set([problem_[i][j] for i in range(PROBLEM_LEN) for j in range(len(problem_[i])) if genome[i]]))"""

def mutationsol(genome):
    """This function just adds one random set to the genome, this is because the current genome isn't yet
    a solution, so we need to add sets(or swap)"""
    point= random.choice([ind for ind in range(len(genome)) if not genome[ind]])
    return genome[:point]+(not genome[point],)+genome[point+1:]

def mutation(genome,toRemove):
    # MUTATION THAT IS BASED ON A # OF ELEMENTS TO REMOVE, AND USING CERTAIN HEURISTICS I WEIGTHED PROBABILITIES OF SWAPPING
    # REMOVING ONLY, ADDING ONLY ETC., THIS WORK COULD BE DONE IN A BETTER WAY THO, MUTATING GENOMES THAT AREN'T SOLUTIONS
    # IN A SMARTER WAY AND ADDING TO THEM ONLY A SET OR SWAPPING, BUT NOT JUST REMOVING, THAT WOULDN'T MAKE SENSE
    # NOW THIS MUTATIONS TRIES TO WORK IN A SMART WAY REMOVING A TON OF ELEMENTS AT THE BEGINNING BECAUSE THERE WE ARE JUST DOING
    # EXPLORATION, AND AT THE END MOSTLY SWAPS BECAUSE WE TRY TO DO EXPLOITATION
    #points to remove
    points= [random.choice([ind for ind in range(PROBLEM_LEN) if genome[ind]]) for _ in range(choice([toRemove-1,toRemove],1,p=[0.2,0.8])[0] if toRemove>0 else toRemove)]
    #points to add
    points.append(random.choice([ind for ind in range(PROBLEM_LEN) if not genome[ind]]))
    return tuple([not genome[ind] if ind in points else genome[ind] for ind in range(PROBLEM_LEN)])

In [ ]:
def gen_population():
    """Generate a population"""
    return [tuple(random.choice([True,False]) for _ in range(PROBLEM_LEN)) for __ in range(POPULATION_SIZE)]
# DEFINITION OF FUNCTIONS USED
fitness_used=fitness_bonus_genome
xover_used=cross_overuni
# GEN POPULATION, GIVE IT FITNESS VALUES AND SORT IT
population_=tuple(sorted([(genome,fitness_used(genome)) for genome in gen_population()],key=lambda a: a[1],reverse=True))
# PRINT POPULATION BY FITNESS VALUES
print([_[1] for _ in population_])

In [ ]:
def search(population__,fitness_function: Callable,xover: Callable):
    """starting from a population try for NUM_GENERATIONS to get a new population"""
    end=False
    for g in range(NUM_GENERATIONS):
        # GET THE WEIGTHS FOR THE PARENT SELECTION, I CHOSE A DISTRIBUTION PROPORTIONAL
        # BASED ON FITNESS, SO THE FITTER AN INDIVIDUAL IS RELATIVELY TO OTHER ONES THE
        # HIGHER ARE HIS CHANCES TO BE PICKED
        offspring,minfit=set(),population__[POPULATION_SIZE-1][1]
        weigths=[-minfit+population__[_][1]+1 for _ in range(POPULATION_SIZE)]
        weigths=[_/sum(weigths) for _ in weigths]
        while len(offspring) < OFFSPRING_SIZE:
            # MUTATION
            if random.random()<0.3:
                # GET A PARENT AND MUTATE IT
                parent=select_parent(population__,weigths,k=1)[0]
                o=mutation(parent[0],ceil((-(-PROBLEM_SIZE-PROBLEM_SIZE+parent[1]))*PROBLEM_LEN/(10*MAXREPETITIONS)))
                f=fitness_function(o)
            else:
                # GETS TWO PARENTS AND USE THEM
                parents= sorted(select_parent(population__,weigths,k=2),key=lambda a: a[1],reverse=True)
                o = xover(parents[0][0], parents[1][0])
                f=fitness_function(o)
                # RECOMBINATION + MUTATION
                if random.random()<0.3:
                    o1=mutation(o,ceil((-(-PROBLEM_SIZE-PROBLEM_SIZE+f))*PROBLEM_LEN/(10*MAXREPETITIONS)))
                    f1=fitness_function(o1)
                    if (f1>minfit and f<minfit) or (f1>f and goal_test(o1)):
                        o,f=o1,f1
            while f>=minfit+AVGPERSET/2 and not goal_test(o):
                # THIS INDIVIDUAL IS REALLY CLOSE TO A GOOD SOLUTION, BUT IS NOT A SOLUTION YET
                # AND ITS FITNESS IS ENOUGH TO MAKE IT SURVIVE, SO UNTIL
                # MUTATIONS KEEPS THE INDIVIDUAL IN THAT RANGE AND THE INDIVUAL IS NO A SOLUTION YET
                # TRY AGAIN, A MUTATION COULD BREAK IT OR MAKE IT
                o= mutationsol(o)
                f=fitness_function(o)
            while (o,f) in offspring or (o,f) in population__:
                # NO DUPLICATES ALLOWED SO MUTATE
                o=mutation(o,ceil((-(-PROBLEM_SIZE-PROBLEM_SIZE+f))*PROBLEM_LEN/(10*MAXREPETITIONS)))
                f=fitness_function(o)
            if goal_test(o):
                offspring.add((o,f))
            if f==2*PROBLEM_SIZE:
                end=True
                break
        population__ = tuple(sorted(list(population__) + list(offspring), key=lambda i: i[1], reverse=True)[:POPULATION_SIZE])
        #logging.info(f"At gen {g} we finished with the fitness pop of {[_[1] for _ in population__]}")
        if end:
            break
    return population__

In [ ]:
MAX_GENERATIONS=NUM_GENERATIONS
while population_[0][1]<1.2*PROBLEM_SIZE:
    gen+=NUM_GENERATIONS
    population_=search(population__=population_,xover=xover_used,fitness_function=fitness_used)
    print(f"After #{gen} generations the current population has the following fitness values( 2 * PROBLEM_SIZE IS OPTIMAL FITNESS, SO IN THIS CASE {PROBLEM_SIZE+PROBLEM_SIZE} ) \n{[_[1] for _ in population_]}")
    if gen>=MAX_GENERATIONS and population_[0][1]<1.2*PROBLEM_SIZE:
        # GIVE UP IF NO SOLUTION WITH GOOD BLOAT IS IN THE POPULATION AFTER MAX GENERATIONS
        print(f"Couldn't find a solution with less bloat in #{gen} generations (more than MAX), this is the best solution found so far")
        break

In [ ]:
print(f"Best individual has fitness {population_[0][1]} individual is {solution_sets(population_[0][0])}\nwith weight {weight_sol(population_[0][0])}\nand it is a VALID SOLUTION? {goal_test(population_[0][0])}\nbecause this genome covers {len(set([problem_[i][j] for i in range(len(population_[0][0])) for j in range(len(problem_[i])) if population_[0][0][i]]))} numbers\nwith {sum(population_[0][0])} sets taken")
print(f"Found in #{gen} generations")